In [ ]:
import torch as t
from torch import nn
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import CIFAR10
import numpy as np
from torch  import optim
import torchvision.utils as vutil
from tensorboard_logger import Logger
import torchvision as tv

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tensorboard_logger import configure, log_value
configure("tensorboard_log/runs/ft")

In [ ]:
class Config:
    lr=0.0002
    nz=100# 噪声维度
    image_size=64
    image_size2=64
    nc=3# 图片三通道
    ngf=64 #生成图片
    ndf=64 #判别图片
    gpuids=2
    beta1=0.5
    batch_size=256
    max_epoch=12# =1 when debug
    workers=8
    clamp_num=0.01# WGAN 截断大小
    
opt=Config()

In [ ]:
# 训练数据集
dataset= ImageFolder('/home/x/data/pre/train_new/nouse2/',
                transform=transforms.Compose([transforms.Scale(opt.image_size),
                                              transforms.RandomCrop(opt.image_size) ,
#                                               transforms.RandomSizedCrop()
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.5]*3,[0.5]*3)
                                             ]))
# 什么惰性加载，预加载，多线程，乱序  全都解决 
dataloader=t.utils.data.DataLoader(dataset,opt.batch_size,True,num_workers=opt.workers)


# 验证数据集
val_dataset=ImageFolder('/home/x/data/pre/test_new/nouse',
                transform=transforms.Compose(\
                                             
                                             [transforms.Scale(opt.image_size),
                                              transforms.RandomCrop(opt.image_size) ,
#                                               transforms.RandomSizedCrop()
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.5]*3,[0.5]*3)
                                             ]))

val_dataloader=t.utils.data.DataLoader(val_dataset,64,True,num_workers=opt.workers)

In [ ]:
# model from GAN

class ModelD(nn.Module):
    def __init__(self,ngpu):
        super(ModelD,self).__init__()
        self.ngpu=ngpu
        self.model=nn.Sequential()
        self.model.add_module('conv1',nn.Conv2d(opt.nc,opt.ndf,4,2,1,bias=False))
        self.model.add_module('relu1',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv2',nn.Conv2d(opt.ndf,opt.ndf*2,4,2,1,bias=False))
        self.model.add_module('bnorm2',nn.BatchNorm2d(opt.ndf*2))
        self.model.add_module('relu2',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv3',nn.Conv2d(opt.ndf*2,opt.ndf*4,4,2,1,bias=False))
        self.model.add_module('bnorm3',nn.BatchNorm2d(opt.ndf*4))
        self.model.add_module('relu3',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv4',nn.Conv2d(opt.ndf*4,opt.ndf*8,4,2,1,bias=False))
        self.model.add_module('bnorm4',nn.BatchNorm2d(opt.ndf*8))
        self.model.add_module('relu4',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv5',nn.Conv2d(opt.ndf*8,1,4,1,0,bias=False))
#         self.model.add_module('sigmoid',nn.Sigmoid())
    def forward(self,input):
        gpuids=None
        if self.ngpu:
            gpuids=range(self.ngpu)
        return nn.parallel.data_parallel(self.model,input, device_ids=gpuids)
netd=ModelD(opt.gpuids)


In [ ]:
# define my classify model
class FinetuneModel(nn.Module):
    def __init__(self, pretrained_model, ngpu=opt.gpuids):
        self.ngpu = ngpu
        super(FinetuneModel,self).__init__()
        self.features = pretrained_model
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 4 * 4, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 2))      
        
    def forward(self, x):
        gpuids = None
        if self.ngpu:
            gpuids = range(self.ngpu)
        features =  self.features(x).view(-1,512*4*4)#, x, device_ids=gpuids,ou)
        return self.classifier(features)
       # return nn.parallel.data_parallel(self.classifier, features, device_ids=gpuids)

## tools to use for validate
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
        
def accuracy(output, target, topk=(1,)):
    """
    Computes the precision@k for the specified values of k
    @param output: score
    @param target: label
    
    """
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def val(model): 
    model.eval()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    for ii,data in enumerate(val_dataloader):
        input,label=data 
        val_input = Variable(input, volatile = True).cuda()
        val_label = Variable(label.type(t.LongTensor), volatile = True).cuda()
        score = model(val_input)
        acc = accuracy(score.cpu().data,label)[0][0]
        val_loss = t.nn.functional.cross_entropy(score,val_label)
        n = input.size()[0]
        acc_meter.update(acc,n)
        loss_meter.update(val_loss.data,n)
        if acc_meter.count>3000:break # val only 3000pictures one time
    model.train()
    return acc_meter, loss_meter


In [ ]:
if hasattr(netd.model,'conv5'):del netd.model.conv5
model = FinetuneModel(netd)

optimizerFC=optim.Adam([{'params':model.features.parameters(),'lr':0.05*opt.lr},
                        {'params':model.classifier.parameters(),'lr':opt.lr}
                       ], betas = (opt.beta1, 0.999))

input=Variable(t.FloatTensor(opt.batch_size,opt.nc,opt.image_size,opt.image_size2)).cuda()
label=Variable(t.LongTensor(opt.batch_size)).cuda()
criterion = nn.CrossEntropyLoss()

model.cuda()
netd.cuda()
criterion.cuda()


In [ ]:
# 训练
# if hasattr(netd.model,'conv5'):del netd.model.conv5
# model = FinetuneModel(netd)
# model.load_state_dict(t.load('class_50_model.nph'))
step = 0
print ii
max_acc = 93.9
for epoch in xrange(260,280):
    for ii, data in enumerate(dataloader,0):
        try:
            model.zero_grad()
            real,label_=data
            input.data.resize_(real.size()).copy_(real)
            label.data.resize_(input.size()[0]).copy_(label_)
            output=model(input)
            error_real=criterion(output,label)
            error_real.backward()
            D_x=output.data.mean()
            optimizerFC.step()
            
            step += 1
            log_value('error_d',error_real.data[0],step)
            log_value('D_x',D_x,step)
            if ii%10==0 and ii>0:
                    a, l = val(model)
                    if a.avg >max_acc: 
                        max_acc=a.avg
                        t.save(model.state_dict(),'class_current_best_model_%s.nph' %max_acc)
                    log_value('val_loss',l.avg[0],step)
                    log_value('val_acc',a.avg,step)
                    log_value('epoch', epoch)
                    log_value('ii', ii)
        except Exception as e:
            print e
                 
                    

In [ ]:
def val_all(model): 
    '''
    test all pictures in test dataset
    '''
    model.eval()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    for ii,data in enumerate(val_dataloader):
        input,label=data 
        val_input = Variable(input, volatile = True).cuda()
        val_label = Variable(label.type(t.LongTensor), volatile = True).cuda()
        score = model(val_input)
        acc = accuracy(score.cpu().data,label)[0][0]
        val_loss = t.nn.functional.cross_entropy(score,val_label)
        n = input.size()[0]
        acc_meter.update(acc,n)
        loss_meter.update(val_loss.data,n)
         
#     log_value('loss_val', val_loss)
#     log_value('accuracy', acc)
    model.train()
    return acc_meter, loss_meter

### 损失函数下降过程
![a](image/loss_d.png)
### 准确率的变化过程
![a](image/val_acc.png)